## 1. 環境確認

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path

# プロジェクトルート
sys.path.append(os.path.abspath('..'))
from configs.config import *

print("=== Environment Check ===")
print(f"Python version: {sys.version}")
print(f"\nProject root: {Path.cwd().parent}")
print(f"Data dir: {DIR_INPUT}")
print(f"\nData exists: {os.path.exists(DIR_INPUT)}")

In [ ]:
# データファイルの存在確認
train_meta_path = os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'train_meta.csv')
test_meta_path = os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'test_meta.csv')
images_dir = os.path.join(DIR_INPUT, 'images')

print("=== Data Files Check ===")
print(f"✅ train_meta.csv: {os.path.exists(train_meta_path)}")
print(f"✅ test_meta.csv: {os.path.exists(test_meta_path)}")
print(f"✅ images/: {os.path.exists(images_dir)}")

if os.path.exists(images_dir):
    image_files = list(Path(images_dir).glob('*.jpg'))
    print(f"\nTotal images: {len(image_files)}")
    if len(image_files) > 0:
        print(f"Sample: {image_files[0].name}")

## 2. rclone セットアップ

### 2.1 rcloneインストール確認

In [ ]:
# rcloneがインストールされているか確認
!rclone version

### 📝 rcloneインストール方法

rcloneがインストールされていない場合、以下の手順でインストールしてください：

#### Windows
```powershell
# Chocolateyを使う場合
choco install rclone

# または公式サイトからダウンロード
# https://rclone.org/downloads/
```

#### macOS
```bash
brew install rclone
```

#### Linux
```bash
curl https://rclone.org/install.sh | sudo bash
```

### 2.2 Google Drive設定

以下のコマンドでrclone設定を開始します（初回のみ）：

In [ ]:
# rclone設定（ターミナルで実行推奨）
# このセルはコメントアウトしたまま、ターミナルで実行してください

# !rclone config

### 📝 rclone設定手順（初回のみ）

ターミナルで `rclone config` を実行し、以下の手順で設定：

1. `n` (New remote) を選択
2. name: `gdrive` と入力（推奨名）
3. Storage: `drive` (Google Drive) を選択
4. client_id: 空白のままEnter
5. client_secret: 空白のままEnter
6. scope: `1` (Full access) を選択
7. root_folder_id: 空白のままEnter
8. service_account_file: 空白のままEnter
9. Edit advanced config?: `n`
10. Use auto config?: `y`
11. ブラウザで認証
12. Configure as team drive?: `n`
13. `y` (Yes this is OK) で完了
14. `q` (Quit config) で終了

In [ ]:
# rclone設定の確認
!rclone listremotes

✅ `gdrive:` が表示されればOK

## 3. データをGoogle Driveにアップロード

### 3.1 アップロード先の確認

In [ ]:
# Google Driveのディレクトリ構造を確認
# まずルートディレクトリ
!rclone lsd gdrive:

### 3.2 プロジェクトフォルダを作成（初回のみ）

In [ ]:
# Google Drive上にプロジェクトフォルダを作成
PROJECT_FOLDER = "atma22"  # ★好きな名前に変更可能★

# フォルダ作成（既に存在する場合はスキップされる）
!rclone mkdir gdrive:{PROJECT_FOLDER}
!rclone mkdir gdrive:{PROJECT_FOLDER}/data
!rclone mkdir gdrive:{PROJECT_FOLDER}/models
!rclone mkdir gdrive:{PROJECT_FOLDER}/logs

print(f"\n✅ Created folder structure in gdrive:{PROJECT_FOLDER}/")

In [ ]:
# フォルダ構造確認
!rclone lsd gdrive:{PROJECT_FOLDER}

### 3.3 データアップロード（初回のみ、時間がかかります）

⚠️ **注意**: 画像データは数GBあるため、初回アップロードに30分〜1時間かかります

In [ ]:
# データサイズ確認（アップロード前）
import shutil

data_size_gb = shutil.disk_usage(DIR_INPUT).used / (1024**3)
print(f"Data directory size: {data_size_gb:.2f} GB")
print(f"\n⚠️ Upload time estimate: {data_size_gb * 2:.0f}-{data_size_gb * 4:.0f} minutes")

In [ ]:
# データアップロード（初回のみ）
# syncコマンド: ローカルとリモートを同期（差分のみアップロード）

print("=== Uploading data to Google Drive ===")
print(f"Source: {DIR_INPUT}")
print(f"Destination: gdrive:{PROJECT_FOLDER}/data/raw/input/")
print("\n⏳ Starting upload... (this may take 30-60 minutes)\n")

!rclone sync {DIR_INPUT} gdrive:{PROJECT_FOLDER}/data/raw/input/ --progress --transfers 4 --checkers 8

In [ ]:
# アップロード確認
print("=== Verifying upload ===")
!rclone ls gdrive:{PROJECT_FOLDER}/data/raw/input/atmaCup22_metadata/
print("\n")
!rclone size gdrive:{PROJECT_FOLDER}/data/raw/input/images/

### 3.4 sample_submission.csvもアップロード

In [ ]:
# sample_submission確認
sample_submission_path = os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'sample_submission.csv')

if os.path.exists(sample_submission_path):
    print(f"✅ sample_submission.csv exists")
    
    # 内容確認
    df_sample = pd.read_csv(sample_submission_path)
    print(f"\nShape: {df_sample.shape}")
    print(df_sample.head())
else:
    print("⚠️ sample_submission.csv not found")

## 4. Gitリポジトリ準備

### 4.1 .gitignore確認

In [ ]:
# .gitignore確認
gitignore_path = Path.cwd().parent / '.gitignore'

if gitignore_path.exists():
    print("✅ .gitignore exists")
    print("\nContents:")
    with open(gitignore_path, 'r', encoding='utf-8') as f:
        print(f.read())
else:
    print("⚠️ .gitignore not found")
    print("\nRecommended .gitignore:")
    print("""
# Data
data/
!data/.gitkeep

# Models
models/
!models/.gitkeep

# Logs
logs/
!logs/.gitkeep

# Python
__pycache__/
*.pyc
*.pyo
*.pyd
.Python

# Jupyter
.ipynb_checkpoints/

# IDE
.vscode/
.idea/
    """)

### 4.2 Git初期化（初回のみ）

In [ ]:
# Git状態確認
%cd ..
!git status

### 📝 Git初期化手順（初回のみ）

Gitリポジトリが未初期化の場合、以下をターミナルで実行：

```bash
# プロジェクトルートで
git init
git add src/ configs/ notebooks/ requirements.txt README.md
git commit -m "Initial commit"

# GitHubリポジトリ作成後
git remote add origin https://github.com/YOUR_USERNAME/atma_22_ca.git
git branch -M main
git push -u origin main
```

In [ ]:
# ソースコードのみをコミット（データは除外）
# ターミナルで実行推奨

# !git add src/ configs/ notebooks/ requirements.txt
# !git commit -m "Prepare for Colab training"
# !git push

## 5. ローカルで小規模テスト実行

Colabで実行する前に、ローカルで小規模データでテストします。

In [ ]:
# データ読み込み（サンプルのみ）
import sys
sys.path.append('..')  # プロジェクトルート

from src.util import Logger
from src.model_resnet_knn import ModelResNet50KNN
from configs.config import *

df_train = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'train_meta.csv'))
df_test = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'test_meta.csv'))

# 小規模サンプル（100件のみ）
df_train_sample = df_train.head(100).copy()
df_test_sample = df_test.head(20).copy()

print(f"Train sample: {df_train_sample.shape}")
print(f"Test sample: {df_test_sample.shape}")

In [ ]:
# テスト実行（小規模データ）
logger = Logger(path=DIR_LOG)

params = {
    'model_name': 'resnet50',
    'k': 3,
    'threshold': 0.5,
    'min2_threshold': 0.3,
    'batch_size': 16,  # 小さめ
    'num_workers': 0,  # テストなので0
    'use_cache': True,
}

print("=== Creating model for test ===")
model = ModelResNet50KNN(
    run_fold_name='test_local',
    params=params,
    out_dir_name='../models',
    logger=logger
)

print("\n=== Training on small sample ===")
model.train(df_train_sample)

print("\n=== Predicting on small sample ===")
pred = model.predict(df_test_sample)

print(f"\n✅ Test completed!")
print(f"Predictions: {pred['label_id'].value_counts()}")

## 6. Colab実行チェックリスト

以下をすべて確認してからColabで実行してください：

In [ ]:
import subprocess

checklist = []

# 1. rclone設定
try:
    result = subprocess.run(['rclone', 'listremotes'], capture_output=True, text=True)
    has_gdrive = 'gdrive:' in result.stdout
    checklist.append(("✅" if has_gdrive else "❌", "rclone設定完了（gdrive:）", has_gdrive))
except:
    checklist.append(("❌", "rclone設定完了（gdrive:）", False))

# 2. データアップロード
try:
    result = subprocess.run(['rclone', 'ls', f'gdrive:{PROJECT_FOLDER}/data/raw/input/atmaCup22_metadata/'], 
                          capture_output=True, text=True, timeout=10)
    has_data = 'train_meta.csv' in result.stdout
    checklist.append(("✅" if has_data else "❌", f"データアップロード完了（gdrive:{PROJECT_FOLDER}/data/）", has_data))
except:
    checklist.append(("❌", f"データアップロード完了（gdrive:{PROJECT_FOLDER}/data/）", False))

# 3. Git push
try:
    result = subprocess.run(['git', 'remote', '-v'], capture_output=True, text=True, cwd='..')
    has_remote = 'origin' in result.stdout
    checklist.append(("✅" if has_remote else "❌", "GitHubリポジトリ設定完了", has_remote))
except:
    checklist.append(("❌", "GitHubリポジトリ設定完了", False))

# 4. ローカルテスト
test_model_exists = os.path.exists('../models/test_local')
checklist.append(("✅" if test_model_exists else "⚠️", "ローカルテスト実行完了", test_model_exists))

# 5. .gitignore
gitignore_exists = os.path.exists('../.gitignore')
checklist.append(("✅" if gitignore_exists else "⚠️", ".gitignore設定", gitignore_exists))

print("\n" + "="*60)
print("【Colab実行チェックリスト】")
print("="*60)

for icon, item, status in checklist:
    print(f"{icon} {item}")

all_ok = all([status for _, _, status in checklist[:3]])  # 最初の3つは必須

print("\n" + "="*60)
if all_ok:
    print("🎉 準備完了！Google Colabで実行できます")
    print("\n次のステップ:")
    print("1. notebooks/exp_knn_colab.ipynb をGoogle Colabで開く")
    print("2. GITHUB_REPO, GDRIVE_DATA_PATH, GDRIVE_BACKUP_PATH を設定")
    print(f"   - GDRIVE_DATA_PATH = \"gdrive:{PROJECT_FOLDER}/data\"")
    print(f"   - GDRIVE_BACKUP_PATH = \"gdrive:{PROJECT_FOLDER}\"")
    print("3. ランタイムタイプをGPUに変更")
    print("4. セクション1から順に実行")
else:
    print("⚠️ まだ準備が完了していません")
    print("\n❌マークの項目を完了させてください")
print("="*60)

## 7. Colab実行時の設定値メモ

Colabで実行する際に必要な設定値を記録しておきます：

In [ ]:
# GitHubリポジトリURL取得
try:
    result = subprocess.run(['git', 'remote', 'get-url', 'origin'], 
                          capture_output=True, text=True, cwd='..')
    github_url = result.stdout.strip()
except:
    github_url = "（未設定）"

print("="*60)
print("【Colab実行時の設定値】")
print("="*60)
print("\nexp_knn_colab.ipynb の以下の変数を設定してください:\n")
print(f'GITHUB_REPO = "{github_url}"')
print(f'GDRIVE_DATA_PATH = "gdrive:{PROJECT_FOLDER}/data"')
print(f'GDRIVE_BACKUP_PATH = "gdrive:{PROJECT_FOLDER}"')
print("\nPROJECT_NAME = \"atma_22_ca\"")
print("="*60)

# クリップボードにコピー用
print("\n📋 以下をコピーしてColabに貼り付けてください:\n")
print(f"""GITHUB_REPO = "{github_url}"
GDRIVE_DATA_PATH = "gdrive:{PROJECT_FOLDER}/data"
GDRIVE_BACKUP_PATH = "gdrive:{PROJECT_FOLDER}"
PROJECT_NAME = "atma_22_ca"
""")

## まとめ

### ✅ 完了した作業
1. rclone設定（Google Drive接続）
2. データアップロード（ローカル → Google Drive）
3. Gitリポジトリ準備
4. ローカル動作確認
5. Colab実行準備完了チェック

### 🚀 次のステップ
1. Google Colabで `notebooks/exp_knn_colab.ipynb` を開く
2. ランタイムタイプをGPUに変更
3. 上記の設定値を入力
4. セクション1から順に実行

### 💡 Tips
- **初回のみ時間がかかる**: データアップロード（30-60分）
- **2回目以降は高速**: rcloneが差分のみ同期
- **Colab実行後**: 結果がGoogle Driveに自動バックアップされる
- **ローカル同期**: `rclone sync gdrive:{PROJECT_FOLDER}/models ./models` で取得